### Get predictions from final models (same main final models used in the rest of the paper)

In [1]:
import gc
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense, Dropout
from keras import optimizers, regularizers, losses

from keras.models import Model
from keras import backend as K
from keras.callbacks import CSVLogger
from keras import metrics

import scipy
import datetime 

import h5py
import keras

import sys

from sklearn.model_selection import ParameterGrid
import pickle


import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

K.tensorflow_backend._get_available_gpus()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path_to_configs = "../"
sys.path.append(path_to_configs)
from configs import *
from models import *

path_to_ext_val_predictions = path_to_configs + path_to_ext_val_results
phens = ["CERAD","BRAAK","PLAQUES","TANGLES","ABETA_IHC","TAU_IHC"]

In [3]:
MTL_final_final_model = pickle.load(open(path_to_configs+path_to_final_chosen_models + "MTL/final.p", "rb" ) )
baselines_final_final_model = pickle.load(open(path_to_configs+path_to_final_chosen_models + "MLP_baselines/final.p", "rb" ) )

path_to_final_models = path_to_configs+final_models_save_path+"models/"
path_to_x_data = path_to_configs+path_to_ext_val_data_folder + "processed/all_human/ge_pca/"

MTL_phenotype_output_mapping = {}

### MTL

In [1]:
for f in os.listdir(path_to_x_data):
    print(f)
    X = np.loadtxt(path_to_x_data+f, delimiter="\t")
    print(X.shape)
    path_to_preds = "%spredictions/%s/MTL/%s/"%(path_to_ext_val_predictions, f, MTL_final_final_model)


    for i in range(101):
        print(i)

        fname = MTL_final_final_model
        path_to_model = path_to_final_models + "MTL/ACT_MSBBRNA_ROSMAP_PCA/%s/%i/200.hdf5"%(MTL_final_final_model,i)


        MTL_model = keras.models.load_model(path_to_model, custom_objects={"ordloss_cur_params": ordloss(0), \
                    "ignorenans_mse": ignorenans_mse, "cat_acc": ignorenans_categorical_accuracy(0), \
                    "ignorenans_scaled_mse": ignorenans_scaled_mse})

        preds = MTL_model.predict(X)

        for j,layer in enumerate(MTL_model.layers[-len(phenotypes):]):
            MTL_phenotype_output_mapping[layer.name[:-4]] = j

        if not os.path.isdir(path_to_preds):
            os.makedirs(path_to_preds)

        pred_df = pd.DataFrame(np.hstack([preds[MTL_phenotype_output_mapping[phenotype]] for phenotype in phens]), columns=phens)
        pred_df.to_csv("%s/%i.csv"%(path_to_preds,i))

        K.clear_session()
        gc.collect()

# MLPs

In [2]:
for f in os.listdir(path_to_x_data):
    print(f)
    X = np.loadtxt(path_to_x_data+f, delimiter="\t")
    path_to_preds = "%spredictions/%s/MLP_baselines/"%(path_to_ext_val_predictions,f)


    for i in range(10):
        print(i)
        preds = []
        for p in phens:
            path_to_model = path_to_final_models +  "MLP_baselines/ACT_MSBBRNA_ROSMAP_PCA/%s/%s/%i/200.hdf5"%(p, baselines_final_final_model[p], i)

            mod = keras.models.load_model(path_to_model, custom_objects={"ordloss_cur_params": ordloss(0), \
                        "ignorenans_mse": ignorenans_mse, "cat_acc": ignorenans_categorical_accuracy(0), \
                        "ignorenans_scaled_mse": ignorenans_scaled_mse})

            preds.append(mod.predict(X))



        if not os.path.isdir(path_to_preds):
            os.makedirs(path_to_preds)

        pred_df = pd.DataFrame(np.hstack(preds), columns=phens)
        pred_df.to_csv("%s/%i.csv"%(path_to_preds,i))

        K.clear_session()
        gc.collect()